<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/main/17_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install eciespy eth_keys cryptos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 3.9 MB/s 
     |████████████████████████████████| 555 kB 11.4 MB/s 
     |████████████████████████████████| 2.0 MB 44.3 MB/s 
     |████████████████████████████████| 105 kB 55.2 MB/s 
     |████████████████████████████████| 481 kB 55.0 MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.2-cp37-cp37m-linux_x86_64.whl size=1236703 sha256=5fc4b22a49f1881fc6340d4eeb6ebe71c231b1e912790129bd076aced0697f8a
  Stored in directory: /root/.cache/pip/wheels/38/70/71/ca13ea3d36ccd0b3d0ec7d7a4ca67522048d695b556bba4f59
Successfully built cytoolz


In [17]:
import ecies
import eth_keys
import coincurve
import hashlib

In [10]:
privKey1 = ecies.utils.generate_eth_key()
privKey1, type(privKey1)

('0xba17493f6242832e65abc1e01c67a263ec4d465c07bc295f7fdc4ba1fbaacc8b',
 eth_keys.datatypes.PrivateKey)

In [11]:
#this generates a private key
privKey2 = ecies.utils.generate_eth_key()
privKey2, type(privKey2)

('0x40be4d2dd296afdeccca3b1321c59d101daeb0b9c38098678fc7fc80a47c81b9',
 eth_keys.datatypes.PrivateKey)

In [12]:
k1 = coincurve.PrivateKey(privKey1.to_bytes())
k2 = coincurve.PrivateKey(privKey2.to_bytes())

p1 = coincurve.PublicKey( privKey1.public_key.to_compressed_bytes() )
p2 = coincurve.PublicKey( privKey2.public_key.to_compressed_bytes() )

In [28]:
shared12 = p2.multiply(k1.secret)
shared21 = p1.multiply(k2.secret)
shared11 = p1.multiply(k1.secret)
shared22 = p2.multiply(k2.secret)

In [27]:
shared12.format().hex(),shared21.format().hex()

('031dcf7e6adfdeb0cc4234d18e05bb16396508fcc4004e8a7688fa045ce8f63ad1',
 '031dcf7e6adfdeb0cc4234d18e05bb16396508fcc4004e8a7688fa045ce8f63ad1')

In [29]:
shared22.format().hex(),shared11.format().hex()

('03b16bd32d129f919ac494c0a1d0ac71ebc6514da1f928e84c666016118e9b63c4',
 '028dbbcf13a3cf1e5fa86e0d3911b457dc22ac97255d3a9908fd20391b577c23b3')

In [18]:
shared_aes_key=hashlib.sha256(shared1.format()).digest()

In [21]:
len(shared_aes_key),len(shared1.format())

(32, 33)

In [25]:
len(ecies.aes_encrypt(shared_aes_key,shared_aes_key))

64